<a href="https://colab.research.google.com/github/nutapol97/Python-for-DS-AI_Nutapol_T./blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import default_rng
from numpy import linalg
from sklearn.metrics import accuracy_score,classification_report
import random, math
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
def train_test(X,Y,size):
  from numpy.random import default_rng
  rng = default_rng()
  SizeTrain=int((size*X.shape[0]))
  random_np=rng.choice(X.shape[0], size=SizeTrain, replace=False)
  X_test=X[random_np,:]
  y_test=Y[random_np,]
  Y_train=np.delete(Y, random_np,axis=0)
  X_train=np.delete(X, random_np,axis=0)
  Y_train=Y_train.reshape(Y_train.shape[0],)
  y_test=y_test.reshape(y_test.shape[0],)
  return X_train,X_test,Y_train,y_test
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class
    def fit(self, X, y):
        self.n_classes = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self.grow_tree(X, y)
    def best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None  
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes
            num_right = num_parent.copy()
            for i in range(1, m):
                c = int(classes[i - 1])
              
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes))
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes))
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr
    def grow_tree(self, X, y, depth=0):
          num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes)]
          predicted_class = np.argmax(num_samples_per_class)
          node = Node(predicted_class=predicted_class)
          if depth < self.max_depth:
              idx,  thr = self.best_split(X, y)
              if idx is not None:
                  indices_left = X[:, idx] < thr
                  X_left, y_left = X[indices_left], y[indices_left]
                  X_right, y_right = X[~indices_left], y[~indices_left]
                  node.feature_index = idx
                  node.threshold = thr
                  node.left = self.grow_tree(X_left, y_left, depth + 1)
                  node.right = self.grow_tree(X_right, y_right, depth + 1)
          return node
    def acc_decisiton_trees(self,X_test,y_test):
      ypredic=model.predict(X_test)
      acc=accuracy_score(y_test, ypredic)
      return acc

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test(X,y,0.7)

In [ ]:
class RandomForest:
    def __init__(self,typeCalculation, B, bootstrap_ratio, with_no_replacement=True):
        self.B = B
        self.bootstrap_ratio = bootstrap_ratio
        self.with_no_replacement = with_no_replacement
        self.typeCalculation=typeCalculation
        if self.typeCalculation=='Scratch':
          self.max_depth = 10
          self.models = [DecisionTree(self.max_depth) for _ in range(B)]
        if self.typeCalculation=='Sklearn':
          self.tree_params = {'max_depth': 2, 'max_features': 'sqrt'}
          self.models = [DecisionTreeClassifier(**self.tree_params) for _ in range(B)]
        
    def fit(self, X, y): 
        
        m, n = X.shape       
        sample_size = int(self.bootstrap_ratio * len(X))

        xsamples = np.zeros((self.B, sample_size, n))
        ysamples = np.zeros((self.B, sample_size))

        xsamples_oob = []  
        ysamples_oob = []
        for i in range(self.B):
            oob_idx = []
            idxes = []
            for j in range(sample_size):
                idx = random.randrange(m)
                if (self.with_no_replacement):
                    while idx in idxes:
                        idx = random.randrange(m)
                idxes.append(idx)
                oob_idx.append(idx)
                xsamples[i, j, :] = X[idx]
                ysamples[i, j] = y[idx]
            mask = np.zeros((m), dtype=bool)
            mask[oob_idx] = True
            xsamples_oob.append(X[~mask])
            ysamples_oob.append(y[~mask])
        oob_score = 0
        for i, model in enumerate(self.models):
            
            _X = xsamples[i]
            _y = ysamples[i]
            model.fit(_X, _y)

            _X_test = np.asarray(xsamples_oob[i])
            _y_test = np.asarray(ysamples_oob[i])
            yhat = model.predict(_X_test)
            oob_score += accuracy_score(_y_test, yhat)
            print(f"Tree {i}", accuracy_score(_y_test, yhat))
        self.avg_oob_score = oob_score / len(self.models)
        print("Average out of bag score")
        print(self.avg_oob_score)
    def predict(self, X): #<---X_test
        #make prediction and return the probabilities
        predictions = np.zeros((self.B, X.shape[0]))
        for i, model in enumerate(self.models):
            yhat = model.predict(X)
            predictions[i, :] = yhat
        return stats.mode(predictions)[0][0]

In [ ]:
model = RandomForest(typeCalculation='Sklearn',B=5, bootstrap_ratio=0.8)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(classification_report(y_test, yhat))

Tree 0 0.4444444444444444
Tree 1 1.0
Tree 2 0.8888888888888888
Tree 3 0.6666666666666666
Tree 4 0.7777777777777778
Average out of bag score
0.7555555555555554
              precision    recall  f1-score   support

           0       0.97      0.94      0.96        34
           1       0.84      0.95      0.89        38
           2       0.97      0.85      0.90        33

    accuracy                           0.91       105
   macro avg       0.92      0.91      0.92       105
weighted avg       0.92      0.91      0.91       105

